In [7]:
import os
import time
import datetime
import warnings
import json

import pandas as pd
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.keys import Keys

In [10]:
def get_driver(debug=False):
        ff_options = Options()
        if not debug:
            ff_options.add_argument("--headless")
            ff_options.add_argument('--no-sandbox')
        else:
            ff_options.add_argument("--window-size=1366,768")
            ff_options.add_argument('--no-sandbox')

        ff_options.add_argument("--disable-notifications")
        ff_options.add_argument("--disable-extensions")
        ff_options.add_argument('--disable-gpu')
        ff_options.add_argument("--lang=en-US")
#         ff_options.add_experimental_option("excludeSwitches", ["enable-automation"])
#         ff_options.add_experimental_option('useAutomationExtension', False)

        ff_options.add_argument('--disable-blink-features=AutomationControlled')
        service = Service(executable_path='../config/geckodriver.exe')
        driver = webdriver.Firefox(service=service, options=ff_options)
        return driver

In [12]:
driver = get_driver(debug=True)

In [13]:
url = 'https://www.slickcharts.com/sp500'
driver.get(url)

In [29]:
stock_table = driver.find_elements(By.XPATH, '//table')[0]
table_header  = stock_table.find_element(By.XPATH, './/thead')
columns = []
for col in table_header.find_elements(By.XPATH, './/th[@scope="col"]'):
    columns.append(col.text)

table_body = stock_table.find_element(By.XPATH, './/tbody')
rows = table_body.find_elements(By.XPATH, './/tr')
stock_info = []
for row in rows:
    rows_atts = row.find_elements(By.XPATH, './/td')
    row_dict = {}
    for att, col in zip(rows_atts,columns):
        row_dict[col] = att.text
    stock_info.append(row_dict)

In [35]:
df = pd.DataFrame(stock_info)
df['Portfolio%'] = df['Portfolio%'].str.replace('%','').astype(float)
df.to_csv('./data/SP500_weightage.csv')